# Preprocessign data sets

## 1. Clean Up Pavlopoulos(2020)
- 공개 데이터셋, Pavlopoulos et al,. 2020. Toxicity Detection: Does Context Really Matter?.
- 이모지, URL, \t,\s와 같은 tag 만 제거하였음.

In [1]:
!wget https://raw.githubusercontent.com/ipavlopoulos/context_toxicity/master/data/CAT_LARGE/gc.csv gc.csv

import pandas as pd
import re

--2021-07-09 01:04:13--  https://raw.githubusercontent.com/ipavlopoulos/context_toxicity/master/data/CAT_LARGE/gc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3697430 (3.5M) [text/plain]
Saving to: ‘gc.csv’

gc.csv              100%[===================>]   3.53M  10.3MB/s    in 0.3s    

2021-07-09 01:04:14 (10.3 MB/s) - ‘gc.csv’ saved [3697430/3697430]

--2021-07-09 01:04:14--  http://gc.csv/
Resolving gc.csv (gc.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘gc.csv’
FINISHED --2021-07-09 01:04:14--
Total wall clock time: 0.8s
Downloaded: 1 files, 3.5M in 0.3s (10.3 MB/s)


In [2]:
df = pd.read_csv('gc.csv')
df.head()

,id,text,parent,label,api
0,100030037.40314.40314,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have h...,0,0.1323
1,100104685.102437.102437,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0,0.0314
2,100189373.112824.112824,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0,0.2229
3,100203140.40393.40393,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the...,0,0.0559
4,100214962.61084.61084,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firefo...,0,0.0539


In [3]:
df.label.value_counts()

0    9849
1     151
Name: label, dtype: int64

In [10]:
def remove_emoji_gc(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    string = emoji_pattern.sub(r'', string)

    comment_number = re.compile("[0-9]+.")
    string = comment_number.sub(r'',string)
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    # url = re.compile('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')
    url = re.compile(regex)
    string = url.sub(r'',string)
    string = re.compile("[']{2,3}").sub(r"'",string)
    string = re.compile("[\'|]{1}").sub(r"'",string)
    string = re.compile("[\']{2,}").sub(r"",string)
    string = re.compile(r"\s+").sub(" ", string).strip()
    return string

In [ ]:
for n, (text, parent, label) in enumerate(zip(df['text'], df['parent'], df['label'])):
  parent = remove_emoji_gc(parent)
  text = remove_emoji_gc(text)
  print(f'Q {n} : {repr(parent)}\nA {n} : {repr(text)}\n L {label}\n')

In [ ]:
for n, (text, parent) in enumerate(zip(df[df['label'] == 1]['text'], df[df['label']==1]['parent'])):
  print(f'Q {n} : {parent}\nA {n} : {text}\n')

In [11]:
df['text_clean'] = df['text'].apply(remove_emoji_gc)
df['parent_clean'] = df['parent'].apply(remove_emoji_gc)

df.head()

,id,text,parent,label,api,text_clean,parent_clean
0,100030037.40314.40314,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have h...,0,0.1323,"BTW, I see no ""attack"" here. What I see is som...",I also feel its important for StuRat to have h...
1,100104685.102437.102437,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0,0.0314,I realise... I just want to urge caution and a...,A main reason this project was started was to ...
2,100189373.112824.112824,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0,0.2229,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion..."
3,100203140.40393.40393,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the...,0,0.0559,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the ...
4,100214962.61084.61084,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firefo...,0,0.0539,I like the additional info in the info box. It...,The larger box displays fine in Mozilla Firefo...


In [ ]:
for n, (text, parent) in enumerate(zip(df[df['label'] == 1]['text_clean'], df[df['label']==1]['parent_clean'])):
  print(f'Q {n} : {parent}\nA {n} : {text}\n')

In [12]:
df_clean = df[['parent_clean','text_clean','label']]
df_clean.rename({'parent_clean':'parent','text_clean':'text'},axis=1,inplace=True)
df_clean.to_csv('gc_clean.csv')


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### NER by spacy
- fine-tuning GPT Neo 활용 문장 생성시, NER을 Keyword로 사용하려는 계획.(시간 부족으로 실제 문장 생성은 하지 못함)

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [27]:
import spacy
from spacy.tokenizer import Tokenizer


In [56]:
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

def ner(text):
  doc = nlp(text)
  for ent in doc.ents:
    if ent:
      return ','.join([ent.text, ent.label_])
    else :
      return 'None'


In [59]:
df_clean['parent_ner'] = df_clean['parent'].apply(ner)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
df_clean['text_ner'] = df_clean['text'].apply(ner)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
df_clean.to_csv('gc_clean_ner.csv')

In [62]:
from google.colab import files
files.download('gc_clean_ner.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## Clean up Qian(2019), Gab & Reddit data
- Pavlopoulos(2020) 데이터와 유사하게 전처리 수행하였으며. 추가로 @tag로 user를 tag한 부분은 삭제, #tag keyword 부분은 남겨두었음.


In [1]:
!wget https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/master/data/gab.csv gab.csv
!wget https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/master/data/reddit.csv raddit.csv

import numpy as np
import pandas as pd
import re

def check_text(df,ranges):
  for i in range(ranges[0],ranges[1]):
    print(df.iloc[i,0])
    print(str(df.iloc[i,1]))
    print(df.iloc[i,2])
    print(df.iloc[i,3])
    print('\n\n')

--2021-07-08 15:43:25--  https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/master/data/gab.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9161771 (8.7M) [text/plain]
Saving to: ‘gab.csv’

gab.csv             100%[===================>]   8.74M  20.0MB/s    in 0.4s    

2021-07-08 15:43:26 (20.0 MB/s) - ‘gab.csv’ saved [9161771/9161771]

--2021-07-08 15:43:26--  http://gab.csv/
Resolving gab.csv (gab.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘gab.csv’
FINISHED --2021-07-08 15:43:26--
Total wall clock time: 0.8s
Downloaded: 1 files, 8.7M in 0.4s (20.0 MB/s)
--2021-07-08 15:43:26--  https://raw.githubusercontent.com/jing-qian/A-Benchmark-Dataset-for-Learning-

In [2]:
class Regex_pattern():
  def __init__(self):
    self.emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    self.comment_number = re.compile("[0-9]+.")
    self.url = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    self.user = re.compile(r"@[a-zA-Z]+")


def remove_emoji(string, regex_pattern):

    string = regex_pattern.emoji_pattern.sub(r'', string)
    string = regex_pattern.comment_number.sub(r'',string)
    string = regex_pattern.url.sub(r'',string)
    # user @ tag, # tag 해시태그는 좀 생각을 해봐야 할듯
    string = regex_pattern.user.sub(r'',string)
    # string = re.compile(r"#[a-zA-Z]+").sub(r'',string)
    return string


In [3]:
regex_pattern = Regex_pattern()

In [6]:
remove_emoji(df_rg.loc[692,'text'], regex_pattern)

' The anti-fa punks are planning to gather in cities nationwide for the mid term elections to cause havoc, chaos and attack Trump supporters.   WE CAN LONGER REMAIN ON THE SIDELEINES WATCHING. WE HAVE TO MEET THEM HEAD ON   #MAGA #KAG #GabFam #BritFam\xa0#SpeakFreely\xa0#BuildTheWall\xa0#DrainTheSwamp\xa0#DeepState\xa0 #SpyGate\xa0#Winning#USA\xa0#WeThePeople #WWGGA\xa0   \xa0         _Angel  \xa0 \xa0\xa0\xa0\xa0\xa0 -Warrior -Girl-    \xa0 _Girl   _Beast       \xa0   \xa0#Gabonservatives,\xa0#MCGA\xa0                  -is-shit             _Focus\xa0        \xa0      _patriot                                  _usa   \n \tWhy?\n \t\tLOL what a dumb cunt\n'

In [5]:
df_gab = pd.read_csv('gab.csv')
df_reddit = pd.read_csv('reddit.csv')

df_rg = pd.concat([df_gab,df_reddit], ignore_index=True)
# df_rg.dropna(inplace=True) # nan for non-toxic label
print(df_rg.isnull().sum(),df_rg.shape)
df_rg.duplicated().sum()

# context가 있는 comment만 남김
df_rg = df_rg[~(df_rg.hate_speech_idx == '[1]')][['text','hate_speech_idx']]
df_rg.head()
df_rg.shape

id                    0
text                  0
hate_speech_idx    1829
response           1829
dtype: int64 (16845, 4)


(11682, 2)

In [ ]:
for i,j in zip(df_rg['text'][700:750], df_rg['hate_speech_idx']):
  print(f'\n{j}\n{i}\n\n')

### Qian(2019) 데이터 문장 분리 로직
- 2차례 이상 이어지는 User간 Conversation 중, 혐오 발언으로 간주될 수 있는 Comment 번호를 라벨링 하였음(Figure 2의 예시 참고).
- Hate Speech로 label된 문장을 중심으로, 그 이전 Comment를 맥락 text로 추출하였음.
- 하나의 Conversation에 여러개의 Hate Speech로 label이 있는 경우, Hate Speech label 수만큼 분리.
- Hate Speech로 label된 문장 이전 맥락이 지나치게 짧은 경우(예를 들어 한단어), 맥락 정보가 포함되지 않는 문장이 되어버리기 때문에, 적어도 5개 단어 토큰(stop word 제외) 보다 커질 때 까지 이전 comment를 포함시킴.

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

import spacy
from spacy.tokenizer import Tokenizer

# 맥락 commnet의 token을 카운트 할 때, stop word, punct를 걸러내기 위해서 사용
nlp = spacy.load("en_core_web_trf")
tokenizer = Tokenizer(nlp.vocab)

In [8]:
def token_count(doc):
  cnt = 0
  for token in tokenizer(doc): # spacy tokenizer
    # print(t)
    if (token.is_stop == False) & (token.is_punct == False):
      cnt +=1
  return cnt

def concat_doc(doc_list):
  if len(doc_list) < 2:
    return doc_list
  else :
    return ['. '.join(reversed(doc_list))]

# def set_text_parent(comments, idx):
def set_text_parent(x, regex_pattern):
  comments = x[0]
  idx = x[1]

  comments = remove_emoji(comments, regex_pattern)
  comments = comments.replace('\t','').split('\n')

  doc = []
  for i in comments:
    i = re.compile(r"\s+").sub(" ", i).strip() # too many space -> single space
    doc.append(i)

  doc = doc[:-1]

  text = []
  parent = []
  label = []

  # for non-toxic comments
  if str(idx) == 'nan':
    text.append(doc[-1])
    max_doc = len(doc)
    cnt = 0
    for i in range(max_doc-2, -1,-1):
      cnt += token_count(doc[i])
      parent.append(doc[i])
      label.append(0)
      if cnt > 5:
        return concat_doc(parent), text, label
      else : continue
    return concat_doc(parent), text, label

  # set start idx 0
  idx_list = [int(i)-1 for i in re.compile(r'\d{1,2}').findall(idx)]

  # only single comment is toxic & single context commnet
  if idx_list[0] == 1 and len(idx_list) < 2:
    parent.append(doc[idx_list[0]-1])
    text.append(doc[idx_list[0]])
    label.append(1)
    return parent, text, label # return single string sentence list

  # single toxic
  if len(idx_list) < 2:
    text.append(doc[idx_list[-1]])
    label = [1]
    cnt = 0
    for i in range(idx_list[-1]-1, -1, -1): 

      cnt += token_count(doc[i])
      parent.append(doc[i])
      
      if cnt > 5:
        return concat_doc(parent), text, label
      else : continue
    return concat_doc(parent), text, label


  if len(idx_list) > 1:

    for j in idx_list:
      if j == 0:
        continue
      
      text.append(doc[j])
      label = [1]
      cnt = 0
      tmp_parent = []
      for i in range(j-1,-1,-1):
        cnt += token_count(doc[i])
        tmp_parent.append(doc[i])
        if cnt > 5:
          parent.append(concat_doc(tmp_parent)[0])
          break
        else : continue
        parent.append(concat_doc(tmp_parent)[0])

  return parent, text, label


In [9]:
df_rg.head()

,text,hate_speech_idx
1,1. This is what the left is really scared of. ...,[3]
2,1. It makes you an asshole.\n2. \tGive it to a...,[2]
3,1. So they manage to provide a whole lot of da...,[2]
4,"1. Hi there, i,m Keith, i hope you are doing w...",[3]
6,"1. Hi developers, give us a follow for updates...",[3]


In [22]:
df_rg.iloc[10612,0]

"1. I heard from Giant Bomb's Abby Russell that they touched some political subjects that need to be discussed but in a subtle way. I guess this is subtle for her.\n2. \tThanks for reminding me that stupid millenial cunt exists, I used to enjoy GB before she turned whatever good remained in it into shit.\n"

In [15]:
from tqdm.notebook import tqdm

t_p, t_t, t_l = [], [], []

for n, (c,i) in enumerate(zip(df_rg['text'],df_rg['hate_speech_idx'])):
  if n % 1000 == 0:
    print(n)
  tmp = (c,i)
  try :
    p, t, l = set_text_parent(tmp, regex_pattern)
  except :
    print(n)
  # print(p,t,l)
  for sp,st in zip(p, t):
    t_p.append(sp)
    t_t.append(st)
    # print(l)
    t_l.append(l[0])
    # print(t_p,t_t,l)

# for n,(p, t) in enumerate(zip(t_p, t_t)):
  # print(f'{n}\nparent - {p}\ntext - {t}\n')

df_split = pd.DataFrame(list(zip(t_p,t_t,t_l)), columns=['parent','text','label'])

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
10612
11000
11226


In [16]:
df_split

,parent,text,label
0,That literally looks like a monkey. Why are we...,Dumb Cunt,1
1,It makes you an asshole.,"Give it to a soldier who has defend it , then ...",1
2,"So they manage to provide a whole lot of data,...",yup. i cant smoke in a bar because it's harmfu...,1
3,"Hi there, i,m Keith, i hope you are doing well...","wow, what a rude cunt",1
4,"Hi developers, give us a follow for updates an...",LOL you can't even spell it nigger,1
...,...,...,...
14141,"That's almost too ridiculous to be true, but I...",I guess 'cause there are some checks keeping t...,1
14142,You sound triggered. Did your sippy cup run ou...,"Don’t cry too hard into your waifu, faggot!",1
14143,Here's what's going to happen: Broward is magi...,The media will cover for the Dems and call rep...,0
14144,"Yeah tone ""ghost"" from the victory inn is stil...","I dont know exactly how you feel, of course, b...",1


In [17]:
df_split.to_csv('gab_reddit.csv')
from google.colab import files
files.download('gab_reddit.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
df_split[df_split.label == 0]

,parent,text,label
13,,Is this peak redneck,0
17,Apparently some people fear that you will comm...,"My ""journalism"" will probably devolve into eve...",0
59,._Milano: if “men have a hard time right now” ...,Poor stupid cunt doesn't have an once of brain...,0
72,The heroin addict certainly is the author. Tel...,Angela Schmidt Pauly is unable to read analyti...,0
99,Seems loud and clear to me:,Did you mean is not a cunt ? Here is more info...,0
135,Wish POTUSwould unseal Obama’s records. He sho...,The POS was half Saudi. Obama Sr. was Saudi Ar...,0
139,"Ah man, those two are just brilliant, even Dud...",Peter with the hand brake off is as good as yo...,0
147,fuckin skål man. skål to the fuck heil dudes f...,actually im retarded and blind and didn't coun...,0
151,"Vote Democrat! Mobs, not Jobs! Safe spaces, no...",The democrats are the enemy. The republicans a...,0
182,"Swing it where ? There is no blue wave, only a...","I haven't voted in over years. Fuck it, I'm ba...",0


### NER by spacy

In [23]:
# nlp = spacy.load('en_core_web_trf')
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])

def ner(text):
  doc = nlp(text)
  for ent in doc.ents:
    if ent:
      return ','.join([ent.text, ent.label_])
    else :
      return 'None'

df_split['parent_ner'] = df_split['parent'].apply(ner)
df_split['text_ner'] = df_split['text'].apply(ner)

df_split.to_csv('gab_reddit_ner.csv')
from google.colab import files
files.download('gab_reddit_ner.csv') 

Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
df_split.label.value_counts()

1    12822
0     1324
Name: label, dtype: int64

In [ ]:
for n, (c, i) in enumerate(zip(df_rg['text'][450:500],df_rg['hate_speech_idx'][450:500])):
  print(n,'\n',c, i,'\n')

# Prepare Training Data with Generated Data by GPT-neo
- Pavlopoulos, Qian 데이터를 기반으로 GPT neo를 fine-tuning 하였고. 문장을 생성하였음.
- 시간 절약을 위해 parent, text 각각 동시에 튜닝 & 문장생성 하였음.
- 개별 csv data file을 통합 하였음. 
```
맥락 문장 = parent
혐오 문장 = text
```

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value

configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp/dataset'),
    'path_colab_data' : Path('./dataset')
    }
})

In [4]:
configs.path['list_csv'] = [f for f in configs.path['path_drive'].glob('*.csv')]
configs

{'path': {'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv')],
  'path_colab_data': PosixPath('dataset'),
  'path_drive': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset')}}

In [5]:
df_p = pd.read_csv('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv', index_col=0)
df_t = pd.read_csv('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv', index_col=0)

In [6]:
df_p.head()

,parent,text,label,parent_gen
0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...
1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...
2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...
3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...
4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...


In [7]:
df_t.head()

,parent,text,label,text_gen
0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,"BTW, I see no ""attack"" here. What I see is so..."
1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,I realise... I just wanted to make sure you h...
2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I should have stressed that this should be a ...
3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,Thanks to Lysy for suggesting this. They're b...
4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,I like the additional info in the info box. I...


In [9]:
sum([ p!=t for p,t in zip(df_p.parent, df_t.parent)]) # identical
sum([ p!=t for p,t in zip(df_p.label, df_t.label)]) 

0

In [8]:
configs

{'path': {'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv')],
  'path_colab_data': PosixPath('dataset'),
  'path_drive': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset')}}

In [10]:
df_merged = df_p.merge(df_t[['parent','text_gen']], how='inner', on=['parent'])

In [12]:
df_merged.to_csv(configs.path['path_drive'] / 'generated_dataset.csv')

## Combination of Generated Text Data
- GTP-neo를 통해 생성된 parent, text를 조합.
- 각 개별 데이터 조합으로 Electra model trining.
- Kaggle data로 일반화 성능 평가
  1. parent(O) - text(O)
  2. parent(G) - text(O)
  3. parent(O) - text(G)
  4. parent(G) - text(G)
  5. parent(O+G) - text(O+G)
  - G : generated, O: original

In [26]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

from pathlib import Path
import numpy as np
import pandas as pd


class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value


configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp'),
    'path_colab_data' : Path('./dataset')
    }
})

configs.path['dataset'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*') if 'generated_dataset.csv' in str(f)][0]
configs.path['list_csv'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*.csv')]

Mounted at /content/gdrive


In [27]:
configs

{'path': {'dataset': PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
  'list_csv': [PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gc_clean_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/gab_reddit_ner.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/parent_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/text_gen_data_v2.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/generated_dataset.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/jig_test.csv'),
   PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset/jig_all.csv')],
  'path_colab_data': PosixPath('dataset'),
  '

In [28]:
df_g1 = pd.read_csv([f for f in configs.path['list_csv'] if 'parent_gen_data_v2.csv' in str(f)][0])

In [49]:
df_g1[df_g1['parent'].duplicated()]

,Unnamed: 0,parent,text,label,parent_gen
4183,4185,How about ?,"Looks legit, but I thought we generally avoide...",0,How about?
4341,4343,See this video of a Wikimania address by resea...,m:Global message delivery/Spam,0,See this video of a different day where a cop...
4701,4703,Sounds good to me.,Im glad to hear it. My intent really isn't to ...,0,Sounds like a lot of work to me.
5346,5348,— Martin ( ·,"Sorry about that. Thanks for fixing it up, guys.",0,—
6054,6057,Thank you.,And it's apparent that I made a repetitive err...,0,Thank you. I will remove it tomorrow because ...
...,...,...,...,...,...
23273,13745,[removed],About the same time you became a retard.,1,[removed]
23375,13847,[deleted],You’re retarded,1,[deleted]
23483,13955,[removed],So asks the nigger kike xD,1,[removed]
23549,14021,[deleted],Your mom is a cunt. You should stop talking to...,0,[deleted]


In [50]:
df_g1[df_g1['parent'].duplicated()].iloc[-2,2]

'Your mom is a cunt. You should stop talking to her.'

In [51]:
df_g2 = pd.read_csv([f for f in configs.path['list_csv'] if 'text_gen_data_v2.csv' in str(f)][0])

In [53]:
df_g2[df_g2['parent'].duplicated()]

,Unnamed: 0,parent,text,label,text_gen
4183,4185,How about ?,"Looks legit, but I thought we generally avoide...",0,"Looks legit, but I thought we generally avoid..."
4341,4343,See this video of a Wikimania address by resea...,m:Global message delivery/Spam,0,m:Global message delivery/Spam
4701,4703,Sounds good to me.,Im glad to hear it. My intent really isn't to ...,0,Im glad to hear it. My intent here is to prom...
5346,5348,— Martin ( ·,"Sorry about that. Thanks for fixing it up, guys.",0,Sorry about that! ) I also updated the descri...
6054,6057,Thank you.,And it's apparent that I made a repetitive err...,0,And it's apparent from his face that he's an ...
...,...,...,...,...,...
23273,13745,[removed],About the same time you became a retard.,1,"About the author, not much. -"
23375,13847,[deleted],You’re retarded,1,You’re retarded.
23483,13955,[removed],So asks the nigger kike xD,1,"So now you claim to be a liberal, retard. You..."
23549,14021,[deleted],Your mom is a cunt. You should stop talking to...,0,Your mom is a cunt


In [56]:
df_merged = pd.concat([df_g1,df_g2], axis=1)

In [57]:
sum([i==j for i,j in zip(df_merged.iloc[:,1], df_merged.iloc[:,6])]) # identical

23674

In [58]:
df_merged.head()

,Unnamed: 0,parent,text,label,parent_gen,Unnamed: 0,parent,text,label,text_gen
0,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,I also feel its important for StuRat to have ...,0,I also feel its important for StuRat to have h...,"BTW, I see no ""attack"" here. What I see is som...",0,"BTW, I see no ""attack"" here. What I see is so..."
1,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,A main reason this project was started was to...,1,A main reason this project was started was to ...,I realise... I just want to urge caution and a...,0,I realise... I just wanted to make sure you h...
2,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I'd go for that. A lot of the most common are...,2,"I'd go for that. (By the way, not all deletion...",I should have stressed that this should be a t...,0,I should have stressed that this should be a ...
3,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,(edit conflict with Piotrus) I agree with the...,3,(edit conflict with Piotrus) I agree with the ...,Thanks to Lysy for shortening and copy editing...,0,Thanks to Lysy for suggesting this. They're b...
4,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,The larger box displays fine in Mozilla Firef...,4,The larger box displays fine in Mozilla Firefo...,I like the additional info in the info box. It...,0,I like the additional info in the info box. I...


In [59]:
df_merged = df_merged.iloc[:,[1,2,3,4,9]]
df_merged

In [62]:
df_gen = df_merged[['text_gen','parent_gen','label']].rename({'parent_gen':'parent','text_gen':'text'}, axis=1)
df_gen

,text,parent,label
0,"BTW, I see no ""attack"" here. What I see is so...",I also feel its important for StuRat to have ...,0
1,I realise... I just wanted to make sure you h...,A main reason this project was started was to...,0
2,I should have stressed that this should be a ...,I'd go for that. A lot of the most common are...,0
3,Thanks to Lysy for suggesting this. They're b...,(edit conflict with Piotrus) I agree with the...,0
4,I like the additional info in the info box. I...,The larger box displays fine in Mozilla Firef...,0
...,...,...,...
23669,I guess 'cause there are some checks keeping ...,"That's almost too ridiculous to be true, but ...",1
23670,"Don’t cry too hard into your waifu, faggot.",You sound triggered. Did you get the link to ...,1
23671,The media will cover for you. You've just edi...,Here's what's going to happen: Broward is mag...,0
23672,"I dont know exactly how you feel, of course, ...","Yeah tone ""ghost"" from the victory inn is sti...",1


In [63]:
df_ori = df_merged[['text','parent','label']]
df_ori

,text,parent,label
0,"BTW, I see no ""attack"" here. What I see is som...",I also feel its important for StuRat to have h...,0
1,I realise... I just want to urge caution and a...,A main reason this project was started was to ...,0
2,I should have stressed that this should be a t...,"I'd go for that. (By the way, not all deletion...",0
3,Thanks to Lysy for shortening and copy editing...,(edit conflict with Piotrus) I agree with the ...,0
4,I like the additional info in the info box. It...,The larger box displays fine in Mozilla Firefo...,0
...,...,...,...
23669,I guess 'cause there are some checks keeping t...,"That's almost too ridiculous to be true, but I...",1
23670,"Don’t cry too hard into your waifu, faggot!",You sound triggered. Did your sippy cup run ou...,1
23671,The media will cover for the Dems and call rep...,Here's what's going to happen: Broward is magi...,0
23672,"I dont know exactly how you feel, of course, b...","Yeah tone ""ghost"" from the victory inn is stil...",1


1. text(Ori) - parent(ori)

In [75]:
condition = 'ori-ori'
df_ori.to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

2. text(Ori) - parent(Gan)

In [76]:
condition = 'ori-gan'
pd.concat([ df_ori['text'],df_gen[['parent','label']] ], axis=1).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

3. text(Gan) - parent(Ori)

In [77]:
# text gen, parant ori
condition = 'gan-ori'
pd.concat([ df_gen['text'],df_ori[['parent','label']] ], axis=1).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

4. text(Gan) - parent(Gan)

In [68]:
condition = 'gan-gan'
df_ori.to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

5. text(Ori+Gan) - parent(Ori+Gan)

In [80]:
condition = 'all-merged'
pd.concat([ df_gen, df_ori ], axis=0, ignore_index=True).to_csv(configs.path['path_drive'] / 'dataset' / f'combination-{condition}.csv')

# Kaggle - jigsaw dataset
### for benchmark
- 훈련된 모델을 평가할 수 있는 맥락을 포함하는 혐오발언 데이터셋이 존재하지 않음.
- Kaggle - jigsaw dataset을 활용, 맥락이 None으로 제공될 때 모델의 분류성능을 측정.
- 모델의 일반화 성능!

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [4]:
from pathlib import Path
import numpy as np
import pandas as pd

import shutil
import zipfile

In [5]:
class MyConfig(dict):
  def __getattr__(self, name): return self[name]
  def __setattr__(self, name, value): self[name] = value


configs = MyConfig({'path':{
    'path_drive' : Path('/content/gdrive/MyDrive/toxic_nlp'),
    'path_colab_data' : Path('./dataset')
    }
})

configs.path['dataset'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*') if 'generated_dataset.csv' in str(f)][0]
configs.path['list_csv'] = [f for f in (configs.path['path_drive'] / 'dataset').glob('*.csv')]


src = [ f for f in (configs.path['path_drive'] / 'dataset' / 'jigsaw').glob('*') if '.DS_' not in str(f) ]
# print(src)

# copy dir tree to colab
for f in src:
  try :
    shutil.copy(str(f), str(f.name) )
  except Exception as e:
    print(e)
    pass

configs.path['path_colab_data'] = Path('.')

f_list = [f for f in configs.path['path_colab_data'].glob('*') if '.zip' in str(f)]
print(f_list)

for ff in f_list:
  with zipfile.ZipFile(ff, 'r') as z_ref:
    z_ref.extractall(str(Path('.')))

[PosixPath('train.csv.zip'), PosixPath('test_labels.csv.zip'), PosixPath('sample_submission.csv.zip'), PosixPath('test.csv.zip')]


In [6]:
# load train & test csv files
files = [f for f in configs.path['path_colab_data'].glob('*.csv') if '.zip' not in str(f)]
jig_train = pd.read_csv([str(f) for f in files if 'train' in str(f)][0])
jig_test = pd.read_csv([str(f) for f in files if 'test' in str(f)][0])
jig_test_label = pd.read_csv([str(f) for f in files if 'test_label' in str(f)][0])

In [7]:
# test label merge 
jig_test = jig_test.merge(jig_test_label, how='inner', on=['id'])
# -1 label 제거
jig_test = jig_test[jig_test.toxic != -1]

jig_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [9]:
jig_test.toxic.value_counts(), jig_train.toxic.value_counts()

(0    57888
 1     6090
 Name: toxic, dtype: int64, 0    144277
 1     15294
 Name: toxic, dtype: int64)

In [10]:
jig_test['parent'] = ''
jig_test_set = jig_test.reset_index()
X_test = jig_test_set[['comment_text','parent']]
y_test = jig_test_set['toxic']

In [13]:
configs.path['path_drive'] / 'dataset' / 'jig_test.scv'

PosixPath('/content/gdrive/MyDrive/toxic_nlp/dataset')

In [24]:
jig_test_set[['comment_text','parent','toxic']].to_csv(configs.path['path_drive'] / 'dataset' / 'jig_test.csv')

In [16]:
jig_train['parent'] = ''
jig_train_set = jig_test.reset_index()

In [19]:
jig_all = pd.concat([ jig_train_set, jig_test_set ], axis=0, ignore_index=True)
jig_all

,index,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,parent
0,5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,
1,7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,
2,11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,
3,13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,
4,14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...
127951,153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0,
127952,153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0,
127953,153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0,
127954,153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0,


In [22]:
jig_all.toxic.value_counts()

0    115776
1     12180
Name: toxic, dtype: int64

In [25]:
jig_all[['comment_text','parent','toxic']].to_csv(configs.path['path_drive'] / 'dataset' / 'jig_all.csv')